In [35]:
# ✅ Librerías estándar
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ✅ OpenCV para preprocesamiento de imágenes
import cv2 as cv

# ✅ Librerías de TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.regularizers import l2  # ¡Importante para regularización!

# ✅ Librería para división de datos
from sklearn.model_selection import train_test_split



In [36]:
# ✅ Rutas de Datos
ruta_train_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_test_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_train_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ruta_test_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'

In [37]:
# ✅ Cargar datos de entrenamiento
df_train = pd.read_csv(ruta_train_csv)
df_test = pd.read_csv(ruta_test_csv)

# ✅ Mostrar las primeras filas
print(df_train.head())
print(df_test.head())

      id  clip_count
0  30001          11
1  30002           2
2  30003          26
3  30004          41
4  30005          49
      id
0  45001
1  45002
2  45003
3  45004
4  45005


In [38]:
df_train.describe()

,id,clip_count
count,15000.000000,15000.000000
mean,37500.500000,37.290133
std,4330.271354,21.922691
min,30001.000000,0.000000
25%,33750.750000,18.000000
50%,37500.500000,37.000000
75%,41250.250000,56.000000
max,45000.000000,75.000000


In [39]:
# ✅ Corregir las rutas de las imágenes en los DataFrames
df_train['image_path'] = df_train['id'].apply(lambda x: os.path.join(ruta_train_img, f"clips-{x}.png"))
df_test['image_path'] = df_test['id'].apply(lambda x: os.path.join(ruta_test_img, f"clips-{x}.png"))

In [40]:
# ✅ Dividir en entrenamiento y validación (80% entrenamiento, 20% validación)
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)

# ✅ Mostrar tamaños
print(f"Tamaño de entrenamiento: {len(train_df)}")
print(f"Tamaño de validación: {len(val_df)}")

Tamaño de entrenamiento: 12000
Tamaño de validación: 3000


In [41]:
# ✅ Función mejorada para cargar y preprocesar imágenes
def load_and_preprocess_image(image_path):
    """
    Carga una imagen en escala de grises, la redimensiona y la normaliza.
    Si la imagen no existe o no puede cargarse, devuelve una imagen vacía.
    """
    img = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    if img is None:
        print(f"⚠️ No se pudo cargar la imagen: {image_path}")
        return np.zeros((128, 128))  # Imagen vacía para mantener la consistencia
    
    img = cv.resize(img, (128, 128))  # Redimensionar a 128x128
    img = img / 255.0  # Normalizar a valores entre 0 y 1
    return img

# ✅ Cargar imágenes y etiquetas con manejo de errores
X_train = np.array([load_and_preprocess_image(path) for path in df_train['image_path']])
y_train = df_train['clip_count'].values

X_val = np.array([load_and_preprocess_image(path) for path in val_df['image_path']])
y_val = val_df['clip_count'].values

# ✅ Agregar un canal extra para la compatibilidad con CNN
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]

# ✅ Mostrar formas de los datos
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_train shape: (15000, 128, 128, 1), y_train shape: (15000,)
X_val shape: (3000, 128, 128, 1), y_val shape: (3000,)


In [ ]:
# ✅ Modelo simplificado y corregido
model = Sequential([
    Input(shape=(128, 128, 1)),  # Capa de entrada corregida
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # Regularización L2
    Dropout(0.3),  # Ligera reducción en Dropout
    Dense(1)  # Capa de salida para regresión
])

# ✅ Compilar el modelo con un learning rate ajustado
model.compile(
    optimizer=Adam(learning_rate=0.0005),  # Ajuste fino del learning rate
    loss='mean_squared_error',
    metrics=[RootMeanSquaredError()]
)

# ✅ Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.00001,
    verbose=1
)

# ✅ Entrenar el modelo SIN Data Augmentation (por ahora)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,  # Mantener tamaño moderado
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 126s 528ms/step - loss: 247.2716 - root_mean_squared_error: 14.8164 - val_loss: 29871.8027 - val_root_mean_squared_error: 172.8342 - learning_rate: 5.0000e-04
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 127s 538ms/step - loss: 99.0751 - root_mean_squared_error: 9.9444 - val_loss: 19475.9453 - val_root_mean_squared_error: 139.5557 - learning_rate: 5.0000e-04
Epoch 3/50
 11/235 ━━━━━━━━━━━━━━━━━━━━ 1:58 528ms/step - loss: 84.0313 - root_mean_squared_error: 9.1525

In [ ]:
import matplotlib.pyplot as plt

# ✅ Graficar la pérdida y el RMSE
plt.figure(figsize=(12, 5))

# Pérdida
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

# RMSE
plt.subplot(1, 2, 2)
plt.plot(history.history['root_mean_squared_error'], label='Entrenamiento')
plt.plot(history.history['val_root_mean_squared_error'], label='Validación')
plt.title('RMSE durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('RMSE')
plt.legend()

plt.show()

In [ ]:
# ✅ Evaluar en el conjunto de validación
val_loss, val_rmse = model.evaluate(X_val, y_val)
print(f"Validación RMSE: {val_rmse}")

In [ ]:
# ✅ Realizar predicciones en el conjunto de prueba
X_test = np.array([load_and_preprocess_image(path) for path in df_test['image_path']])
X_test = X_test[..., np.newaxis]

predictions = model.predict(X_test)
df_test['clip_count'] = predictions

# ✅ Guardar predicciones
df_test[['id', 'clip_count']].to_csv('submission.csv', index=False)